### Set your credentials

In [2]:
import os
import yaml
from dotenv import load_dotenv

### Set Credentials

In [3]:
load_dotenv()

True

In [4]:

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") ## Put your OpenAI API key here
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") ## Put your Tavily Search API key here
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents'

## Self Ask with Search Agent

This agent breaks down a complex question into a series of simpler questions and uses a search tool to look up answers to the simpler questions in order to answer the original complex question. It's suitable for scenarios where the question is complex and needs to be broken down.

This is the source paper for [Self-Ask](https://ofir.io/self-ask.pdf)




Difference between Chain of Thought and Self Ask

![self-ask.png](../Unit1-Foundation_LLM_Agents/selfask.png)


Let's review [self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search) as it's defined in Langchain.

In [5]:
from langchain_community.tools.tavily_search import TavilyAnswer
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer
from langchain import hub

In [6]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

### Set up your LLM

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-3.5-turbo-0125',
                 temperature=0.3)

### Set up your Search Tools

We will initialize the tools we want to use. This is a good tool because it gives us answers (not documents)

For this agent, only one tool can be used and it needs to be named “Intermediate Answer”

In [8]:
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

### Setup and run your agent

![agent-steps](../Unit1-Foundation_LLM_Agents/key-agent-steps-first-agent.png)


In [9]:
# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [10]:
agent_executor.invoke(
    {"input": "Which year was the founder of Anthropic born in?"}
)



> Entering new AgentExecutor chain...
Could not parse output: Yes.Invalid or incomplete responseFollow up: Who is the founder of Anthropic?Dario Amodei is the founder of Anthropic, a company established in 2021 by former senior members of OpenAI.Follow up: When was Dario Amodei born?Dario Amodei was born in 1983.So the final answer is: 1983

> Finished chain.


{'input': 'Which year was the founder of Anthropic born in?', 'output': '1983'}

In [12]:
agent_executor.invoke(
    {"input": "Who was the president of US when superconductivity were discovered?"}
)



> Entering new AgentExecutor chain...
Yes. 
Follow up: When was superconductivity discovered?Superconductivity was discovered in the conventional sense during the 1950s through the development of two significant theories: the phenomenological Ginzburg–Landau theory in 1950 and the microscopic BCS theory in 1957.Follow up: Who was the president of the US during the 1950s?Harry Truman was the president of the United States during the 1950s, specifically from 1950 to 1952.So the final answer is: Harry Truman

> Finished chain.


{'input': 'Who was the president of US when superconductivity were discovered?',
 'output': 'Harry Truman'}

In [14]:
agent_executor.invoke(
    {"input": "I want to hike the highest mountain in Asia. How long does will it take me to hike it?"}
)



> Entering new AgentExecutor chain...
Could not parse output: No.Invalid or incomplete responseFollow up: Which is the highest mountain in Asia?Mount Everest is the highest mountain in Asia, reaching an elevation of 29,032 feet (8,849 meters).So the final answer is: Mount Everest

> Finished chain.


{'input': 'I want to hike the highest mountain in Asia. How long does will it take me to hike it?',
 'output': 'Mount Everest'}

### Exercise

- Test out the self-ask-with-search agent on your own questions
- Where does it work, where does it fail?
- What are the limitations of this approach?